# Coding Assignment 5: Support Vector Machine Classifier

CS 598 Practical Statistical Learning

2023-11-27

UIUC Fall 2023

**Authors**
* Ryan Fogle
    - rsfogle2@illinois.edu
    - UIN: 652628818
* Sean Enright
    - seanre2@illinois.edu
    - UIN: 661791377

**Contributions**

## Introduction

In this assignment, we implement a Support Vector Machine (SVM) classifier using the Pegasos (Primal Estimated sub-GrAdient SOlver for SVM) algorithm. The Pegasos algorithm itself optimizes SVM parameters using stochastic gradient descent (SGD).

To validate our implementation, we classify two digits of the MNIST dataset.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

## The Pegasos Algorithm

The learning problem for a SVM can be stated as the following minimization problem:
$$
\min_{\beta, \alpha}
\frac{\lambda}{2} \lVert \beta \rVert^2
+ \frac{1}{n} \sum_{i=1}^{n} \left [ 1 - y(x^t \beta + \alpha) \right ]_{+}
$$

In the Pegasos algorithm, this objective function $J$ is iteratively approximated with samples of the training set observations as follows.

$$
J_i(\beta, \alpha) = 
\frac{\lambda}{2} \lVert \beta \rVert^2
+ \frac{1}{n} \sum_{i=1}^{n} \left [ 1 - y_i(x_i^t \beta + \alpha) \right ]_{+}
$$

where $\alpha$ is the intercept and $\beta$ is the p-dimensional coefficient vector.

These two parameters are updated iteratively by using their subgradients to approach a local optimum. These subgradients are

$$
\Delta_t =
\begin{cases}
    \lambda \beta_t - y_i x_i & \text{if} y_i(x_i^t \beta_t + \alpha_t) < 1 \\
    \lambda \beta_t           & \text{otherwise} \\
\end{cases}
$$

$$
\delta_t =
\begin{cases}
    -y_i & \text{if} y_i(x_i^t \beta_t + \alpha_t) < 1 \\
    0    & \text{otherwise} \\
\end{cases}
$$

where $\Delta_t$ and $\delta_t$ are the subgradients at iteration $t$ with respect to $\beta$ and $\alpha$, respectively.

The Pegasos algorithm can be summarized as follows:

1. initialize $\beta = 0_{p \times 1}$, $\alpha_1 = 0$, and $t=0$
2. for epoch = $1, 2, ..., T$, do
    * for $i = 1, 2, …, n$, do
        * $t = t + 1$
        * $\eta_t = \frac{1}{t \, \lambda}$
        * $\beta_{t+1} \Leftarrow \beta_t - \eta_t \, \Delta_t$
        * $\alpha_{t+1} \Leftarrow \alpha_t - \eta_t \, \delta_t$

In [ ]:
def pegasos(X, Y, epochs=20, lam=1):
    # shuffle data
    inds = np.arange(X.shape[0])
    np.random.shuffle(inds)
    X = X[inds]
    Y = Y[inds]

    # intialize parameters
    N = X.shape[0]
    p = X.shape[1]
    t = 0
    alpha = 0
    beta = np.zeros(p).reshape(-1, 1)
    
    # loop through each epoch
    for epoch in range(epochs):

        # loop through each data point
        for i in range(N):
            t = t + 1
            nt = 1 / (t * lam)
            if Y[i] * (np.dot(X[i], beta) + alpha) < 1:
                big_delta = lam * beta - (Y[i]*X[i]).reshape(-1,1)
                little_delta = -Y[i]
            else:
                big_delta = lam * beta
                little_delta = 0
            beta = beta - nt * big_delta
            alpha = alpha - nt * little_delta
    return beta, alpha

## Data

To train a SVM and validate its classification ability, a subset of the MNIST dataset is used. 200 training samples and 600 test samples are used. Only observations with the labels "5" and "6" are used.

In [ ]:
# set random seed
np.random.seed(3434)

# Read in data
train = pd.read_csv('coding5_train.csv')
test = pd.read_csv('coding5_test.csv')

# Create Data matrixes
X_train = train[train.columns[:-1]].copy().values
Y_train = train['Y'].copy().values
X_test = test[test.columns[:-1]].copy().values
Y_test = test['Y'].copy().values

Our SVM expects the labels to be either $-1$ or $1$, so we relabel $5$ as $-1$, and $6$ as 1.

In [ ]:
# Set one class to be -1, the other to 1
# in this case when Y == 5, we set it to -1
# when Y == 6, we set it to 1. 
y_train = np.ones(Y_train.shape[0])
y_train[Y_train == Y_train.min()] = -1

y_test = np.ones(Y_test.shape[0])
y_test[Y_test == Y_test.min()] = -1

## Results

To validate our SVM implementation, we present confusion tables for the training and test datasets, as well as test error.

## Predictions
Here we generate predictions $\hat{y}$ by
$$
\hat{y} =
\begin{cases}
0 &  x^t \beta + \alpha < 0 \\
1 &  x^t \beta + \alpha > 0 \\
\end{cases}
$$

and relabel the predictions as label "5" or "6" accordingly.

In [ ]:
beta, alpha = pegasos(X_train, y_train)
Y_pred = (X_test @ beta + alpha).reshape(-1)

## Confusion Matrices

### Training Dataset

In [ ]:
fig = ConfusionMatrixDisplay(confusion_matrix(Y_test == Y_test.max(), Y_pred > 0))
fig.plot()
plt.show()

### Test Dataset

## Test Error

In [ ]:
acc = accuracy_score(Y_test == Y_test.max(), Y_pred.reshape(-1) > 0)
print(f'Accuracy: {acc:.2f}')
print(f'Test Error: {1 - acc:.2f}')